In [1]:
!pip install tensorflow_addons
import tensorflow as tf
import numpy as np
from sklearn.metrics import accuracy_score
import tensorflow_addons as tfa

     |████████████████████████████████| 1.1 MB 14.5 MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#model = tf.keras.models.load_model("/content/drive/MyDrive/model_best.h5") #best model
#model = tf.keras.models.load_model("/content/drive/MyDrive/model_final.h5") #shared model
#model = tf.keras.models.load_model("/content/drive/MyDrive/adv1/baseline_with_reptile_224.h5")
#model = tf.keras.models.load_model("/content/drive/MyDrive/adv1/gaussian_blur.h5")
model = tf.keras.models.load_model("/content/drive/MyDrive/adv1/baseline_with_reptile_224_ep75.h5")

In [4]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [5]:
input_shape = (256,256,3)
#input_shape_model = (224,224,3)
s=389
inputs = tf.keras.Input(shape=input_shape)
x = tf.keras.layers.Rescaling(1.0 / 255)(inputs)
x = tf.keras.layers.Normalization(mean=[0.485, 0.456, 0.406], variance=[0.229**2, 0.224**2, 0.225**2])(x)
x = tf.keras.layers.RandomCrop(height=224,width=224,seed=s)(x)
#x = tf.keras.layers.Resizing(height = 256,width = 256)(x)
#x = tf.keras.layers.GaussianNoise(stddev = sd,seed =s)(x)
outputs = model(x)
new_model = tf.keras.Model(inputs, outputs)

In [6]:
!unzip /content/drive/MyDrive/test_resized.zip -d /content/test_img # to unzip the training file in drive

Streaming output truncated to the last 5000 lines.
  inflating: /content/test_img/test_resized/ILSVRC2012_val_00045001.JPEG  
  inflating: /content/test_img/test_resized/ILSVRC2012_val_00045002.JPEG  
  inflating: /content/test_img/test_resized/ILSVRC2012_val_00045003.JPEG  
  inflating: /content/test_img/test_resized/ILSVRC2012_val_00045004.JPEG  
  inflating: /content/test_img/test_resized/ILSVRC2012_val_00045005.JPEG  
  inflating: /content/test_img/test_resized/ILSVRC2012_val_00045006.JPEG  
  inflating: /content/test_img/test_resized/ILSVRC2012_val_00045007.JPEG  
  inflating: /content/test_img/test_resized/ILSVRC2012_val_00045008.JPEG  
  inflating: /content/test_img/test_resized/ILSVRC2012_val_00045009.JPEG  
  inflating: /content/test_img/test_resized/ILSVRC2012_val_00045010.JPEG  
  inflating: /content/test_img/test_resized/ILSVRC2012_val_00045011.JPEG  
  inflating: /content/test_img/test_resized/ILSVRC2012_val_00045012.JPEG  
  inflating: /content/test_img/test_resized/ILSVR

In [8]:
image_path = "/content/test_img"
from keras.preprocessing.image import ImageDataGenerator
test_datagen = ImageDataGenerator()

test_set = test_datagen.flow_from_directory(image_path,
                                                 target_size = (256, 256),
                                                 batch_size = 1,
                                                 class_mode = 'categorical',
                                                 shuffle = False)
#test_set_cropped = center_crop_normalize_generator(test_set, 224, mean=[0.485, 0.456, 0.406],
#                                     std=[0.229, 0.224, 0.225])

Found 50000 images belonging to 1 classes.


In [9]:
file_names = test_set.filenames
nb_samples = len(file_names)
print(len(file_names),file_names[2],file_names[20000])

50000 test_resized/ILSVRC2012_val_00000003.JPEG test_resized/ILSVRC2012_val_00020001.JPEG


In [10]:
y = new_model.predict(test_set,steps = np.ceil(nb_samples/1),verbose = 1)

50000/50000 [==============================] - 480s 9ms/step


reptile+baseline 224

In [11]:
y_pred = np.argmax(y, axis = 1)
print(y_pred.shape)
y_true = np.loadtxt("/content/drive/MyDrive/adv1/ILSVRC2012_test_ground_truth.txt").astype(int)
#finding the accuracy
acc = accuracy_score(y_true,y_pred)
print(acc)

(50000,)
0.1708


## When tested on model_best(gaussian blur)

In [ ]:
y = model.predict(test_set,steps = np.ceil(nb_samples/1),verbose = 1)

50000/50000 [==============================] - 489s 10ms/step


In [ ]:
y_pred = np.argmax(y, axis = 1)
print(y_pred.shape)
y_true = np.loadtxt("/content/drive/MyDrive/adv1/ILSVRC2012_test_ground_truth.txt").astype(int)
#finding the accuracy
acc = accuracy_score(y_true,y_pred)
print(acc)

(50000,)
0.0014


## When tested on model_best(baseline)

In [ ]:
y = model.predict(test_set,steps = np.ceil(nb_samples/1),verbose = 1)

In [ ]:
y_pred = np.argmax(y, axis = 1)
print(y_pred.shape)
y_true = np.loadtxt("/content/drive/MyDrive/adv1/ILSVRC2012_test_ground_truth.txt").astype(int)
#finding the accuracy
acc = accuracy_score(y_true,y_pred)
print(acc)

(50000,)
0.26136


var2

In [ ]:
y = new_model.predict(test_set,steps = np.ceil(nb_samples/1),verbose = 1)

50000/50000 [==============================] - 523s 10ms/step


In [ ]:
y_pred = np.argmax(y, axis = 1)
print(y_pred.shape)
y_true = np.loadtxt("/content/drive/MyDrive/adv1/ILSVRC2012_test_ground_truth.txt").astype(int)
#finding the accuracy
acc = accuracy_score(y_true,y_pred)
print(acc)

(50000,)
0.23552


var3

In [ ]:
y = new_model.predict(test_set,steps = np.ceil(nb_samples/1),verbose = 1)

50000/50000 [==============================] - 476s 9ms/step


In [ ]:
y_pred = np.argmax(y, axis = 1)
print(y_pred.shape)
y_true = np.loadtxt("/content/drive/MyDrive/adv1/ILSVRC2012_test_ground_truth.txt").astype(int)
#finding the accuracy
acc = accuracy_score(y_true,y_pred)
print(acc)

(50000,)
0.23552


# When tested on model_final(baseline)

In [ ]:
y = model.predict(test_set_cropped,steps = np.ceil(nb_samples/1),verbose = 1)

50000/50000 [==============================] - 711s 14ms/step


In [ ]:
y_pred = np.argmax(y, axis = 1)
print(y_pred.shape)
y_true = np.loadtxt("/content/drive/MyDrive/adv1/ILSVRC2012_test_ground_truth.txt").astype(int)
#finding the accuracy
acc = accuracy_score(y_true,y_pred)
print(acc)

(50000,)
0.23828


In [ ]:
#used on validation data/test data
def normalize_image(image, mean, std):
    for channel in range(3):
        image[:,:,channel] = (image[:,:,channel] - mean[channel]) / std[channel]
    return image

def center_crop_normalize(img, crop_size,mean,std):
    height, width = img.shape[0], img.shape[1]
    dy, dx = crop_size
    x = (width - dx + 1) // 2
    y = (height - dy + 1) // 2
    return normalize_image(img[y:(y+dy), x:(x+dx), :],mean,std)

def center_crop_normalize_generator(batches, crop_length,mean,std):
    """Take as input a Keras ImageGen (Iterator) and generate random
    crops from the image batches generated by the original iterator.
    """
    while True:
        batch_x, batch_y = next(batches)
        batch_crops = np.zeros((batch_x.shape[0], crop_length, crop_length, 3))
        for i in range(batch_x.shape[0]):
            batch_crops[i] = center_crop_normalize(batch_x[i], (crop_length, crop_length), mean, std)
        yield (batch_crops, batch_y)

# Reptile_model

In [ ]:
rep_model = tf.keras.models.load_model("drive/MyDrive/adv1/reptile")
rep_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 rescaling (Rescaling)       (None, 224, 224, 3)       0         
                                                                 
 normalization (Normalizatio  (None, 224, 224, 3)      0         
 n)                                                              
                                                                 
 resnet50 (Functional)       (None, 5)                 23597957  
                                                                 
Total params: 23,597,957
Trainable params: 23,544,837
Non-trainable params: 53,120
_________________________________________________________________


In [ ]:
for layer in rep_model.layers:
  x= layer.get_config()
  print(x)
  if x['name']== 'resnet50':
    print(len(x['layers']))
    y = layer.get_weights()


{'batch_input_shape': (None, 224, 224, 3), 'dtype': 'float32', 'sparse': False, 'ragged': False, 'name': 'input_1'}
{'name': 'rescaling', 'trainable': True, 'dtype': 'float32', 'scale': 0.00392156862745098, 'offset': 0.0}
{'name': 'normalization', 'trainable': True, 'dtype': 'float32', 'axis': (-1,), 'mean': ListWrapper([0.485, 0.456, 0.406]), 'variance': ListWrapper([0.052441, 0.050176000000000005, 0.050625])}
{'name': 'resnet50', 'layers': [{'class_name': 'InputLayer', 'config': {'batch_input_shape': (None, 224, 224, 3), 'dtype': 'float32', 'sparse': False, 'ragged': False, 'name': 'input_2'}, 'name': 'input_2', 'inbound_nodes': []}, {'class_name': 'ZeroPadding2D', 'config': {'name': 'conv1_pad', 'trainable': True, 'dtype': 'float32', 'padding': ((3, 3), (3, 3)), 'data_format': 'channels_last'}, 'name': 'conv1_pad', 'inbound_nodes': [[['input_2', 0, 0, {}]]]}, {'class_name': 'Conv2D', 'config': {'name': 'conv1_conv', 'trainable': True, 'dtype': 'float32', 'filters': 64, 'kernel_size'

In [ ]:
inputs = rep_model.layers[3].get_layer("input_2").output
outputs = rep_model.layers[3].get_layer("avg_pool").output

In [ ]:
model2 = tf.keras.Model(inputs=inputs, outputs=outputs)
model2.trainable = False
model2.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [ ]:
inputs = tf.keras.Input(shape=(224, 224, 3))
x = tf.keras.layers.Rescaling(1.0 / 255)(inputs)
x = tf.keras.layers.Normalization(mean=[0.485, 0.456, 0.406], variance=[0.229**2, 0.224**2, 0.225**2])(x)
x = model2(x, training=False)
outputs = tf.keras.layers.Dense(1000,activation = "softmax")(x)
new_model = tf.keras.Model(inputs, outputs)
new_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 rescaling (Rescaling)       (None, 224, 224, 3)       0         
                                                                 
 normalization (Normalizatio  (None, 224, 224, 3)      0         
 n)                                                              
                                                                 
 model (Functional)          (None, 2048)              23587712  
                                                                 
 dense (Dense)               (None, 1000)              2049000   
                                                                 
Total params: 25,636,712
Trainable params: 2,049,000
Non-trainable params: 23,587,712
_______________________________________

In [ ]:
input_shape = (224,224,3)
layer_name = "avg_pool"
inputs = tf.keras.Input(input_shape)